# Analyz Your Instagram Followers Without an App

## Introduction

One of the most common curiosities among Instagram users is whether the people they follow are also following them back. There are numerous apps available on the App Store and Google Play Store designed to answer this question. However, the reliability of these apps often leaves users with many doubts and concerns. Considering this, I have conceived a solution that utilizes basic coding and a Google extension, allowing for a reliable and simple approach to uncover this information. In this article, I will share with you how to achieve this.

While integrating Instagram's API could potentially make this process faster and provide real-time data, I will not delve into this aspect in our discussion. This approach aims to offer a secure and straightforward method, steering clear of the complexities and potential trust issues associated with third-party apps.

Two essential components are needed: 
- The IG Follower Export Tool extension developed for Chrome
- A Python editor (I personally use and recommend Jupyter)

## Step 1: Download Followers and Following Data with Extension.

For the first step, the extension installed is used to extract the account's followers and following data.

NOTE: For this to work, one must be logged into an Instagram account from another tab (logging into any account is sufficient; there is no need to log into the account from which the data is being retrieved).

After entering the username or link in the "IG User or URL" section, you can select a delay from the dropdown menu and click on "Start new parsing". The reason for adding a delay is because Instagram prevents scraping. You should set a larger timeout if you encounter issues. The timeout is set in seconds. Then, in the bottom right corner, you can monitor the total number of people and the status in the "Progress" section. When the status changes to 'Finished', you can download the file by clicking the 'save to excel' button.

![Image of Yaktocat](https://miro.medium.com/v2/resize:fit:720/format:webp/1*xtxMp0dWJ53AOuAWlkSWLA.png)

Similarly, by selecting the "Extracting type" value as "Following" instead of "Followers", you can initiate another parsing process. Once this operation is completed, you can download the data into an Excel file by clicking on 'save to excel'.

The headers of the Excel files we download appear as follows: id - userName - fullName - profileUrl - avatarUrl - isVerified - followedByYou. The important columns for us are userName, FullName, and profileURL. We will select these columns in Python to obtain a more meaningful dataframe.

As a result, with the help of a simple extension, we now possess the followers and following information of the Instagram account we wish to check, in the form of an Excel file.

## Step 2: Compare Two Data Files with Python

First, we import our followers and following Excel files into the Python environment using read_excel. Next, we filter out the followers and those we are following based on the 'userName' column. By taking the difference between these two groups, we identify users who don't follow us back.

Then, we select the 'userName', 'fullName', 'profileUrl' columns from this group to create a DataFrame that contains the usernames, full names, and direct clickable links to the profiles of the accounts not following back. This allows us to see the users who are not following us back.

### Please specify the path to your following and followers Excel files in the # Load the Excel files section.

In [ ]:
import pandas as pd
from IPython.display import HTML

# Load the Excel files
followers_df = pd.read_excel('followers.xlsx')
following_df = pd.read_excel('following.xlsx')

# Extract the lists of followers and following based on the 'userName' column
followers_usernames = set(followers_df['userName'])
following_usernames = set(following_df['userName'])

# Find users that you are following but who don't follow you back
not_following_back = following_usernames - followers_usernames

# Create a DataFrame containing information about users not following back
not_following_back_df = following_df[following_df['userName'].isin(not_following_back)]

# Select only the desired columns
result_df = not_following_back_df[['userName', 'fullName', 'profileUrl']]

# Convert URLs to clickable HTML links
def make_clickable(val):
    return f'<a href="{val}" target="_blank">{val}</a>'

# Work with a copy to avoid warning
result_copy_df = result_df.copy()
result_copy_df['profileUrl'] = result_copy_df['profileUrl'].apply(make_clickable)

# Display as HTML
HTML(result_copy_df.to_html(escape=False))